# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# Connect to client libraries

openai = OpenAI()

gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)




In [ ]:
models = ["gpt-5","gemini-2.5-pro", "qwen2.5-coder", "deepseek-coder-v2" ]

clients = {"gpt-5": openai, "gemini-2.5-pro": gemini,"qwen2.5-coder": ollama, "deepseek-coder-v2": ollama,}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [5]:
from system_info import retrieve_system_info, rust_toolchain_info

system_info = retrieve_system_info()
rust_info = rust_toolchain_info()
rust_info

{'installed': False,
 'rustc': {'path': '',
  'version': '',
  'host_triple': '',
  'release': '',
  'commit_hash': ''},
 'cargo': {'path': '', 'version': ''},
 'rustup': {'path': '',
  'version': '',
  'active_toolchain': '',
  'default_toolchain': '',
  'toolchains': [],
  'targets_installed': []},
 'rust_analyzer': {'path': ''},
 'env': {'CARGO_HOME': '',
  'RUSTUP_HOME': '',
  'RUSTFLAGS': '',
  'CARGO_BUILD_TARGET': ''},
 'execution_examples': []}

In [6]:
message = f"""
Here is a report of the system information for my computer.
I want to run a Rust compiler to compile a single rust file called main.rs and then execute it in the simplest way possible.
Please reply with whether I need to install a Rust toolchain to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile Rust code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.
Have the maximum possible runtime performance in mind; compile time can be slow. Fastest possible runtime performance for this platform is key.
Reply with the commands in markdown.

System information:
{system_info}

Rust toolchain information:
{rust_info}
"""

response = openai.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

You do not currently have a Rust toolchain installed. You’ll need to install it.

Simplest install (native on Apple M1)
1) Install rustup (the standard Rust toolchain manager):
```
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
```
2) Load the environment for this shell:
```
source "$HOME/.cargo/env"
```
3) Verify:
```
rustc --version
cargo --version
```

Now, to compile a single main.rs for maximum runtime performance and then run it from Python, use:

Python:
```python
import subprocess

compile_command = [
    "rustc", "main.rs",
    "-C", "opt-level=3",
    "-C", "lto=fat",
    "-C", "codegen-units=1",
    "-C", "target-cpu=native",
    "-C", "panic=abort",
    "-C", "debuginfo=0",
    "-o", "main"
]

compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)

run_command = ["./main"]
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
print(run_result.stdout)
```

Notes:
- These rustc flags prioritize fastest possible runtime on your Apple M1 (AArch64 macOS), accepting slower compile times.

In [7]:
compile_command = [
    "/Users/ed/.cargo/bin/rustc",
    "main.rs",
    "-C", "opt-level=3",
    "-C", "target-cpu=native",
    "-C", "codegen-units=1",
    "-C", "lto=fat",
    "-C", "panic=abort",
    "-C", "strip=symbols",
    "-o", "main",
]

run_command = ["./main"]


## And now, on with the main task

In [8]:
language = "Rust" # or "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [10]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)